In [1]:
import numpy as np
from ml_models import OptimizationModel


In [2]:
# Define random bounds for the variables
bounds = [
    {'name': 'var1', 'type': 'continuous', 'domain': (0, 10)},
    {'name': 'var2', 'type': 'continuous', 'domain': (0, 10)},
    {'name': 'var3', 'type': 'continuous', 'domain': (0, 10)}
]

# Define a random target value
target_value = np.random.randint(1, 1000)

# Define random reagent information
reagent_info = {'reagent1': 20, 'reagent2': 30, 'reagent3': 25, 'reagent4': 50, 'reagent5': 15}  # Assuming volumes in μL

# Define fixed reagents (randomly chosen)
fixed_reagents = [('reagent2', 30), ('reagent3', 25), ('reagent4', 50)]  # Example fixed reagents with volumes

# Initialize the optimization model
opt_model = OptimizationModel(bounds, target_value, reagent_info, fixed_reagents, initial_design_numdata=15, batch_size=3)

# Function to simulate chemistry experiments
def simulate_chemistry(x):
    """
    Simulates a chemistry experiment based on input variables x.
    The function returns a 'result' mimicking an experimental outcome.
    """
    # Simulate some arbitrary chemistry logic that relates x to an outcome
    result = np.sum(x**2, axis=1) * np.random.uniform(0.95, 1.05, size=x.shape[0])
    # Ensure the result is a 2D array with one column
    return result.reshape(-1, 1)

# Generate initial design and simulate experiments to get initial data
X_initial = opt_model.generate_initial_design()
Y_initial = simulate_chemistry(X_initial)

# Initialize the optimizer with initial experimental data
opt_model.initialize_optimizer(X_initial, Y_initial)

print(opt_model.optimizer)
print(opt_model.acquisition)


# Learning loop
number_of_iterations = 15  # Define the number of iterations for the learning loop
for i in range(number_of_iterations):
    # Suggest next locations for experiments
    next_locations = opt_model.suggest_next_locations()
    
    # Simulate experiments at the suggested locations
    Y_new = simulate_chemistry(next_locations)
    
    # Update the optimizer with the new experimental data
    opt_model.update_experiment_data(next_locations, Y_new)
    
    print(f"Iteration {i+1}, Suggested Locations: {next_locations}, Results: {Y_new}")
    
    # Optional: Check if the optimizer has met the target or other stopping criteria
    if opt_model.optimizer.quit:
        print("Stopping criteria met.")
        break

1
MPI
last, <GPyOpt.acquisitions.MPI.AcquisitionMPI object at 0x7fc91992ef70>


AssertionError: 

In [ ]:
# Define random bounds for the variables
bounds = [{'name': 'var1', 'type': 'continuous', 'domain': (0, 10)},
          {'name': 'var2', 'type': 'continuous', 'domain': (0, 10)},
          {'name': 'var3', 'type': 'continuous', 'domain': (0, 10)}]




# Define a random target value
target_value = np.random.randint(1, 1000)

# Define random reagent information
reagent_info = {'reagent1': 20, 'reagent2': 30, 'reagent3': 25, 'reagent4': 50, 'reagent5': 15}  # Assuming volumes in μL

# Define fixed reagents (randomly chosen)
fixed_reagents = [('reagent2', 30), ('reagent3', 25), ('reagent4', 50)]  # Example fixed reagents with volumes

# Initialize the optimization model
opt_model = OptimizationModel(bounds, target_value, reagent_info, fixed_reagents)

# Get initial design samples
initial_design = opt_model.get_initial_design()

# Test the model with random data
num_tests = 5
for i in range(num_tests):
    # Generate random experimental data (just for demonstration purposes)
    X_new = np.random.rand(1, len(bounds))
    Y_new = np.random.rand()  # Random output value
    
    # Update the model with the new experimental data
    opt_model.update_model(X_new, Y_new)
    
# Suggest the next location for experimentation
next_location = opt_model.suggest_next_location()

print("Initial Design:")
print(initial_design)
print("\nNext Location for Experimentation:")
print(next_location)